In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re
%matplotlib inline

Get data and convert it into a pandas dataframe

In [2]:
dir = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/"
train = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/train.json"
test = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/test.json"


data_file = open(train, 'r')
data = json.load(data_file)
data_file.close()

df = pd.DataFrame(data)

In [3]:
print(df.columns)

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address'],
      dtype='object')


Get the features column. Remove - and double \*. Then use \* to split the features and combine all of them into a list. Convert it into a series at the end.

In [4]:
features = df['features'].tolist()
features_list = []
for feature in features:
    temp = []
    if feature != []:
        for f in feature:
            f = f.lower()
            f = re.sub("[-]", "", f)
            if ' * ' in f:    
                f = f.split(' * ')
                for element in f:
                    element = re.sub("[**]", "", element)
                    temp.append(element.strip())
            else:
                temp.append(f.strip())    
    features_list.append(temp)
features = pd.Series(features_list)

In [5]:
print(features.shape)

(49352,)


In [6]:
flattenFeatures = pd.Series([l.strip() for subL in features_list for l in subL])

In [7]:
uniqueFeatures = flattenFeatures.drop_duplicates()
len(uniqueFeatures)

1438

In [8]:
g = flattenFeatures.value_counts()
keepFeatures = g.index

In [9]:
featuresdf = pd.DataFrame(columns = keepFeatures)
featuresdf['features'] = features
featuresdf['listing_id'] = pd.Series(df['listing_id'].tolist())
print(featuresdf.shape)
print(featuresdf.columns)

(49352, 1440)
Index(['elevator', 'hardwood floors', 'cats allowed', 'dogs allowed',
       'doorman', 'dishwasher', 'laundry in building', 'no fee',
       'fitness center', 'prewar',
       ...
       'modern village marvel', 'two private balconies',
       'brand new renovation', 'sparkling clean & bright',
       'huge private backyard', 'last', 'queen & king size bedrooms',
       'parks in area', 'features', 'listing_id'],
      dtype='object', length=1440)


In [10]:
#featuresdf = featuresdf.iloc[1:11,]

** This part puts 1 for the features that exist for that listing **

In [11]:
featuresdfc = featuresdf.copy()

for index, row in featuresdf.iterrows():
    f = list(row[-2])
    for f1 in f:
        featuresdfc.loc[index, f] = 1

In [ ]:
featuresdfc.to_csv(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/featuresdf.csv")
featuresdfc.to_json(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/featuresdf.json")

In [ ]:
df_sub = df[['listing_id', 'bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'interest_level', 'latitude',
       'longitude', 'manager_id', 'photos', 'price',
       'street_address']]

In [29]:
df_sub = pd.merge(df_sub, featuresdfc, on = 'listing_id', how='inner')
df_sub.to_csv(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/features_split.csv")
df_sub.to_json(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/features_split.json")

# Fixing the rows issue by merging the data frames properly

In [3]:
dir = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/"
train = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/train.json"
test = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/test.json"
features_only = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/featuresdf.csv"
train_f_l = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/features_f_l.csv"
loc = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/LocationClusters.csv"

features_only_df = pd.read_csv(features_only, encoding = "ISO-8859-1")
train_df = pd.read_json(train)
loc_df = pd.read_csv(loc)

In [4]:
f_col = features_only_df.columns
t_col = train_df.columns
loc_l = loc_df.columns

In [5]:
f_col = [f for f in f_col if f not in t_col]
f_col.append("listing_id")
features_only_df = features_only_df[f_col]

loc_l = [f for f in loc_l if f not in t_col]
loc_l.append('listing_id')
loc_df = loc_df[loc_l]

In [6]:
df = pd.merge(features_only_df, train_df, on = 'listing_id', how='left')
df = pd.merge(df, loc_df, on = 'listing_id', how='left')

In [7]:
df.to_json(train_f_l)